# Preprocessing

In [52]:
import numpy as np;
import csv;
X = [];
with open('Econ_Freedom_Data.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',');
    for row in csv_reader:
        if row[0] == "2016":
            v = [];
            for i in range(6,36) :
                if (row[i]=="") :
                    v.append(-1.0);
                else :
                    v.append(float(row[i]));
            X.append(v);
i = 0;
while i < np.shape(X)[0] :
    v = X[i];
    if -1.0 in v :
        X = np.delete(X,i,0);
    else :
        i = i+1;
meanv = np.repeat(np.reshape(np.asmatrix(X.mean(0)),(1,np.shape(X)[1])),np.shape(X)[0],axis=0);
stdv = np.repeat(np.reshape(np.asmatrix(X.std(0)),(1,np.shape(X)[1])),np.shape(X)[0],axis=0);
Xs = (X-meanv) / stdv;

# k-Means Clustering (Lester Fan)

# Gaussian Mixture Model (Bill Lee)

# ---- (Anton Maliev)